In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas
%matplotlib inline
from shapely.geometry import Polygon
import warnings
warnings.filterwarnings(action='ignore')
from dataPreparation import data_preparation

df_rent, df_house_price, df_rent_price = data_preparation()

# there are NaN, we replace them with 0
df = df_rent.fillna(0)
df_house_price = df_house_price.fillna(0)
df_rent_price = df_rent_price.fillna(0)

df

In [ ]:
# create a new column with the country name by taking the last word of the column 'title'
splitted = df['title'].str.split()
df['country'] = splitted.str[-1]
df['country'].replace({'Republic':'Czech Republic'}) # replace the 'Republic' with 'Czech Republic'

# drop useless columns
df.drop(columns=['title'], inplace=True)
df.drop(columns=['text-data'], inplace=True)

df

In [ ]:
# rename the columns and replace 'Not specified' prices with 0
df.rename(columns={"desktop-hidden 2": "price", "desktop-hidden 4": "surface"}, inplace=True)

# normal replace not working so, replace the ',' with ''
df['price'] = df['price'].astype(str)
df['price'].replace({'0,':'0'}, regex=True, inplace=True)
df['price'].replace({'1,':'1'}, regex=True, inplace=True)
df['price'].replace({'2,':'2'}, regex=True, inplace=True)
df['price'].replace({'3,':'3'}, regex=True, inplace=True)
df['price'].replace({'4,':'4'}, regex=True, inplace=True)
df['price'].replace({'5,':'5'}, regex=True, inplace=True)
df['price'].replace({'6,':'6'}, regex=True, inplace=True)
df['price'].replace({'7,':'7'}, regex=True, inplace=True)
df['price'].replace({'8,':'8'}, regex=True, inplace=True)
df['price'].replace({'9,':'9'}, regex=True, inplace=True)

df['surface'].replace({'0,':'0'}, regex=True, inplace=True)
df['surface'].replace({'1,':'1'}, regex=True, inplace=True)
df['surface'].replace({'2,':'2'}, regex=True, inplace=True)
df['surface'].replace({'3,':'3'}, regex=True, inplace=True)
df['surface'].replace({'4,':'4'}, regex=True, inplace=True)
df['surface'].replace({'5,':'5'}, regex=True, inplace=True)
df['surface'].replace({'6,':'6'}, regex=True, inplace=True)
df['surface'].replace({'7,':'7'}, regex=True, inplace=True)
df['surface'].replace({'8,':'8'}, regex=True, inplace=True)
df['surface'].replace({'9,':'9'}, regex=True, inplace=True)

# remove not valid data
df = df[df['price'] != "Not specified"]
df = df[df['surface'] != 0]
df = df[df['price'] != 0]

df

In [ ]:
#convert all the prices to € and remove the 'EUR'

# CHF, DKK, PLN, SEK, CZK, BGN, HUF, RON, Iceland
# create multiplier for the prices
multiplier = {'EUR': 1, 'CHF': 1, 'DKK': 0.13, 'PLN': 0.21, 'SEK': 0.09, 'CZK': 0.04, 'BGN': 0.51, 'HUF': 0.0025, 'RON': 0.2, 'ISK': 0.0072, 'NOK': 0.096}

prices = []
for price in df['price']:
    if price.find('EUR') != -1:
        price = price.replace('EUR', '')
        price = float(price) * multiplier['EUR']
    elif price.find('CHF') != -1:
        price = price.replace('CHF', '')
        price = float(price) * multiplier['CHF']
    elif price.find('DKK') != -1:
        price = price.replace('DKK', '')
        price = float(price) * multiplier['DKK']
    elif price.find('PLN') != -1:
        price = price.replace('PLN', '')
        price = float(price) * multiplier['PLN']
    elif price.find('SEK') != -1:
        price = price.replace('SEK', '')
        price = float(price) * multiplier['SEK']
    elif price.find('CZK') != -1:
        price = price.replace('CZK', '')
        price = float(price) * multiplier['CZK']
    elif price.find('BGN') != -1:
        price = price.replace('BGN', '')
        price = float(price) * multiplier['BGN']
    elif price.find('HUF') != -1:
        price = price.replace('HUF', '')
        price = float(price) * multiplier['HUF']
    elif price.find('RON') != -1:
        price = price.replace('RON', '')
        price = float(price) * multiplier['RON']
    elif price.find('ISK') != -1:
        price = price.replace('ISK', '')
        price = float(price) * multiplier['ISK']
    elif price.find('NOK') != -1:
        price = price.replace('NOK', '')
        price = float(price) * multiplier['NOK']
    else:
        price = float(price)
    prices.append(price)
df['price'] = prices

df

In [ ]:
# clean the column 'surface'
df['surface'].replace({' m2':''}, regex=True, inplace=True)
df['surface'] = df['surface'].astype(float)

# calculate the index for every apartment
df['index'] = df['price']/df['surface']

# remove ousiders
df = df[df['index'] < 1000]

# write the new dataframe to a csv file
df.to_csv('result.csv', sep='\t')

In [ ]:
# first plots of the data
df = df[df['index'].isna() == False]
df.groupby(['country'])['index'].mean().plot(legend=True, kind='bar', figsize=(10,5), title='Average index by country', xlabel='Country', ylabel='Average index')

In [ ]:
df.groupby(['country'])['price'].mean().plot(legend=True, kind='bar', figsize=(10,5))

In [ ]:
df_house_price.rename(columns={"Unnamed: 0": "country"}, inplace=True)
df_house_price

In [ ]:
df_rent_price.rename(columns={"Unnamed: 0": "country"}, inplace=True)
df_rent_price

In [ ]:
ax = plt.gca()
  
#use plot() method on the dataframe
df_rent_price.plot( x = 'country', y = '2020', ax = ax, figsize=(10,5), legend=True, kind='bar', title='Average rent price by country', xlabel='Country', ylabel='Average rent price')

In [ ]:
# compare the rent_price and house_price indexes with a bar plot
countries = df_rent_price['country']
rentSeries = df_rent_price['2020']
priceSeries = df_house_price['2020']

df_bar = pd.DataFrame({"Rent":rentSeries,"Price":priceSeries})

ax = df_bar.plot.bar(color=["SkyBlue","IndianRed"], rot=0, title="Index complare\nRent vs. Price", figsize=(10,5))
ax.set_xlabel("country")
ax.set_ylabel("indexes")
plt.show()

In [ ]:
ax = plt.gca()
  
#use plot() method on the dataframe
df_house_price.plot( x = 'country', y = '2020', ax = ax, figsize=(10,5), legend=True, kind='bar', title='Average house price by country', xlabel='Country', ylabel='Average house price')

In [ ]:
# Geoplots, first by year and then larger with last year
plt.rcParams['figure.figsize']=(12,10)
plt.rcParams['font.size']=12

world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
europe=world[world.continent=='Europe']

#Remove Russia from map of Europe
europe=europe[(europe.name!='Russia')]

# Create a custom polygon
polygon = Polygon([(-25,35), (40,35), (40,75),(-25,75)])
poly_gdf = geopandas.GeoDataFrame([1], geometry=[polygon], crs=world.crs)

#Clip polygon from the map of Europe
europe=geopandas.clip(europe, polygon) 

selected_countries=europe[europe.name.isin(list(df_rent_price.country))]
df_rent_price['name'] = df_rent_price['country']
selected_countries=selected_countries.merge(df_rent_price, on='name',how='left')

#Range of Variable you see as map color. Here I select the minimum and maximum of all the years selected.
vmin=selected_countries[['1996','2000','2005','2010','2015','2020']].min().min()
vmax=selected_countries[['1996','2000','2005','2010','2015','2020']].max().max()

#Create subplots with 2 rows and 3 columns
fig,axs=plt.subplots(2,3)
fig.suptitle('Rent indicator price in Europe 1996–2020', fontweight='bold',fontsize=15)
#Adjust space betweeen rows
plt.subplots_adjust(bottom=0.2, top=0.9, hspace=0.25)
axs[0,0]=europe.plot(color='whitesmoke',edgecolor='black',ax=axs[0,0])
selected_countries.plot('1996',cmap='Reds',edgecolor='black',ax=axs[0,0], vmin=vmin, vmax=vmax)
axs[0,0].set_title('1996')
axs[0,0].xaxis.set_visible(False)
axs[0,1]=europe.plot(color='whitesmoke',edgecolor='black',ax=axs[0,1])
selected_countries.plot('2000',cmap='Reds',edgecolor='black',ax=axs[0,1], vmin=vmin, vmax=vmax)
axs[0,1].set_title('2000')
axs[0,1].xaxis.set_visible(False)
axs[0,1].yaxis.set_visible(False)
axs[0,2]=europe.plot(color='whitesmoke',edgecolor='black',ax=axs[0,2])
selected_countries.plot('2005',cmap='Reds',edgecolor='black',ax=axs[0,2], vmin=vmin, vmax=vmax)
axs[0,2].set_title('2005')
axs[0,2].xaxis.set_visible(False)
axs[0,2].yaxis.set_visible(False)
axs[1,0]=europe.plot(color='whitesmoke',edgecolor='black',ax=axs[1,0])
selected_countries.plot('2010',cmap='Reds',edgecolor='black',ax=axs[1,0], vmin=vmin, vmax=vmax)
axs[1,0].set_title('2010')
axs[1,1]=europe.plot(color='whitesmoke',edgecolor='black',ax=axs[1,1])
selected_countries.plot('2015',cmap='Reds',edgecolor='black',ax=axs[1,1], vmin=vmin, vmax=vmax)
axs[1,1].set_title('2015')
axs[1,1].yaxis.set_visible(False)
axs[1,2]=europe.plot(color='whitesmoke',edgecolor='black',ax=axs[1,2])
selected_countries.plot('2020',cmap='Reds',edgecolor='black',ax=axs[1,2], vmin=vmin, vmax=vmax)
axs[1,2].set_title('2020')
axs[1,2].yaxis.set_visible(False)

# add colorbar
cax = fig.add_axes([0.92, 0.2, 0.03, 0.7]) #[left, bottom, width, height]
sm = plt.cm.ScalarMappable(cmap='Reds', norm=plt.Normalize(vmin=vmin, vmax=vmax))
# fake up the array of the scalar mappable.
sm._A = []
lgd=fig.colorbar(sm, cax=cax).set_label('gCO$_2$e/kWh', rotation=0,y=1.05, labelpad=-35)
plt.savefig('Rent_price.jpeg',
 dpi=300)
plt.show()

In [ ]:
fig,ax=plt.subplots()

fig.suptitle('Rent indicator price in Europe 1996–2020', fontweight='bold',fontsize=15)

ax=europe.plot(color='whitesmoke',edgecolor='black',ax=ax)
selected_countries.plot('2020',cmap='Reds',edgecolor='black',ax=ax, vmin=vmin, vmax=vmax, legend=True)
ax.set_title('2020')
ax.xaxis.set_visible(False)

plt.savefig('Rent_price.jpeg', dpi=300)
plt.show()

In [ ]:
selected_countries=europe[europe.name.isin(list(df_house_price.country))]
df_house_price['name'] = df_house_price['country']
selected_countries=selected_countries.merge(df_house_price, on='name',how='left')

#Range of Variable you see as map color. Here I select the minimum and maximum of all the years selected.
vmin=selected_countries[['1996','2000','2005','2010','2015','2020']].min().min()
vmax=selected_countries[['1996','2000','2005','2010','2015','2020']].max().max()

fig,ax=plt.subplots()

fig.suptitle('Rent indicator price in Europe 1996–2020', fontweight='bold',fontsize=15)

ax=europe.plot(color='whitesmoke',edgecolor='black',ax=ax)
selected_countries.plot('2020',cmap='Reds',edgecolor='black',ax=ax, vmin=vmin, vmax=vmax, legend=True)
ax.set_title('2020')
ax.xaxis.set_visible(False)

plt.savefig('Rent_price.jpeg', dpi=300)
plt.show()

In [ ]:
# Booble charts
plt.figure(figsize=(10, 10))

# Scatterplot
plt.scatter(
    x = df.groupby(['country'])['surface'].mean(),
    y = df.groupby(['country'])['price'].mean(), 
    s = df.groupby(['country'])['index'].mean()*100, 
    c = df['country'].groupby(df['country']).size().values, 
    cmap="Accent", 
    alpha=0.6, 
    edgecolors="white", 
    linewidth=2);
 
# Add titles (main and on axis)
plt.yscale('log')
plt.xlabel("Life Expectancy")
plt.ylabel("GDP per Capita")
plt.title("Year 1952")
plt.ylim(0,50000)
plt.xlim(30, 75);

In [ ]:
PRICE_LENGTH = df["price"].values
INDEX_LENGTH = df["index"].values

COUNTRIES = df["country"].values
COUNTRIES_ = np.unique(COUNTRIES)

COLORS = ["#1B9E77", "#D95F02", "#7570B3", "#E7298A", "#66A61E", "#E6AB02", "#1B9E77", "#D95F02", "#7570B3", "#E7298A", "#66A61E", "#E6AB02", "#1B9E77", "#D95F02", "#7570B3", "#E7298A", "#66A61E", "#E6AB02", "#1B9E77", "#D95F02", "#7570B3", "#E7298A", "#66A61E", "#E6AB02", "#1B9E77", "#D95F02", "#7570B3", "#E7298A", "#66A61E", "#E6AB02", "#1B9E77", "#D95F02", "#7570B3", "#E7298A", "#66A61E", "#E6AB02", "#1B9E77", "#D95F02", "#7570B3", "#E7298A", "#66A61E", "#E6AB02", "#1B9E77", "#D95F02", "#7570B3", "#E7298A", "#66A61E", "#E6AB02", "#1B9E77", "#D95F02", "#7570B3", "#E7298A", "#66A61E", "#E6AB02", "#1B9E77", "#D95F02", "#7570B3", "#E7298A", "#66A61E", "#E6AB02", "#1B9E77", "#D95F02", "#7570B3", "#E7298A", "#66A61E", "#E6AB02", "#1B9E77", "#D95F02", "#7570B3", "#E7298A", "#66A61E", "#E6AB02", "#1B9E77", "#D95F02", "#7570B3", "#E7298A", "#66A61E", "#E6AB02", "#1B9E77", "#D95F02", "#7570B3", "#E7298A", "#66A61E", "#E6AB02"]

fig, ax = plt.subplots(figsize=(8,8))
for country, color in zip(COUNTRIES_, COLORS):
    idxs = np.where(COUNTRIES == country)
    # No legend will be generated if we don't pass label=species
    ax.scatter(
        PRICE_LENGTH[idxs], INDEX_LENGTH[idxs], label=country,
        s=50, color=color, alpha=0.7
    )
    
ax.legend();